In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import time
from jax import device_put, devices, config
import jax.experimental.sparse as jsparse
from scipy import sparse
from innov.automaton_exnovation import *
from innov.tree_gen import *
from innov.network_model_1_SDE_jax_obs_front import * 
from innov.simple_model import ODE2Tree_obs_random
import pickle

In [ ]:
"""
Run and save the resuts from Automaton and SDE

"""

dt = .01       # time steps
el = 50, 5000  # size of binary trees: initial chain number of nodes, number of nodes of the following K chains  
samples = 100  # Austomaton and SDE samples
"""
Dinamical parameters
"""

G_in = 40   # incoming agents
r = 0.2     # replication rate
I = 0.8     # Innovation rate

"""
Structural parameters
"""
K=2         # number of branches

tplot = 100        # dt's to save tepmoral variables 
tplot_steps = 5000  # number of temporal variables to save

tmax = tplot*tplot_steps*dt


key = random.PRNGKey(10)
h = []
for rd, ro in [ (0.1, 0.1), (0.3, 0.3), (0.15, 0.35), (0.25, 0.1), (0.8, 0.8)]:       # rest of dynamic parameters: rate of death, rate of obsolescence
        for γ in [0, 0.5, 1.0]:   # rest of sreuctural parameters: connectednes
            γ = float(γ)
            
            # define graph structure
            tree = KTree(*el, K, γ)
            # transform Ady into a sparse matrix
            
            Ady = jsparse.BCOO.from_scipy_sparse(tree.adj)
            Ady.data = Ady.data.astype(jnp.int8)
            
            def init_variables(N, samples):
                """Define an initial condition.

                Parameters
                ----------
                N : int
                    Size of graph.
                samples : int
                    Number of parallel jobs to run.
                """
                inn = jnp.zeros((samples,N), dtype=jnp.bool_)
                obs = jnp.zeros((samples,N), dtype=jnp.bool_)
                adj_obs = jnp.zeros((samples,N), dtype=jnp.bool_)
                sub = jnp.zeros((samples,N), dtype=jnp.bool_)
                n = jnp.zeros((samples,N), dtype=jnp.float32)

                x_inn = el[0]-1
                inn = inn.at[:,x_inn].set(True)
                adj_obs = adj_obs.at[:,0].set(True)
                n = n.at[:,:x_inn+1].set(10)
                sub = sub.at[:,:x_inn+1].set(True)
                return inn, obs, sub, n, adj_obs

            
            """
            setup SDE simulations:
                select one of the modes of:
                    propagate_mode:
                        SDE: Sthocastic Differential Equations
                        ODE: Ordinary Differential equations
                    obs_mode:
                        random: normal noise in the movement of obsolescence
                        average: no noise
                    innov_front_mode:
                        explorer_random: normal noise in the movement of obsolescence
                        explorer: no noise
            """
            
            init_vars, one_loop, run_save = setup_auto_num_int(N = Ady.shape[0],
                                                               r = r,
                                                               rd = rd,
                                                               I = I,
                                                               G_in = G_in,
                                                               Δt = dt,
                                                               ro = ro,
                                                               key = key,
                                                               samples = samples,
                                                               Ady = Ady,
                                                               init_fcn=init_variables, propagate_mode='SDE', obs_mode='random',
                                                               innov_front_mode = 'explorer_random')
            
            """
            setup Automaton simulations:
            """
            init_vars_1, one_loop_1, run_save_1 = setup_auto_sim(N = Ady.shape[0],
                                                               r = r,
                                                               rd = rd,
                                                               I = I,
                                                               G_in = G_in,
                                                               dt = dt,
                                                               ro = ro,
                                                               key = key,
                                                               samples = samples,
                                                               Ady = Ady,
                                                               init_fcn=init_variables, obs_mode='random',
                                                               innov_front_mode = 'explorer')
            # initialize variables
            xinn = jnp.zeros((samples,Ady.shape[0]), dtype=jnp.bool_)
            xobs = jnp.zeros((samples,Ady.shape[0]), dtype=jnp.bool_)
            out_vars = one_loop(0,[key]+list(init_vars)+[xinn, xinn, xobs])
            out_vars_1 = one_loop_1(0,[key]+list(init_vars)+[xinn, xobs])
            
            # viriables to save temporal information
            #    # SDE
            L_int = []
            L_std_int =[]
            N_std_int =[]
            N_int = []
            n_o_int = []
            n_o_std_int = []
            pos_inn_int = []
            pos_obs_int = []
            
            #    # Automaton
            L = []
            L_std =[]
            N_std =[]
            N = []
            n_o = []
            n_o_std = []
            pos_inn = []
            pos_obs = []
            
            for i in range(tplot_steps):
                out_vars = fori_loop(0, tplot, one_loop, out_vars)
                out_vars_1 = fori_loop(0, tplot, one_loop_1, out_vars_1)
                L.append(jnp.sum(out_vars_1[3], axis = 1))
                n_o.append(np.mean(out_vars_1[4][out_vars_1[1]]))
                N.append(jnp.sum(out_vars_1[4], axis = 1))
                n_o_std.append(np.std(out_vars_1[4][out_vars_1[1]]))
                pos_inn.append(np.mean(out_vars_1[1], axis=0))
                pos_obs.append(np.mean(out_vars_1[5], axis=0))
                
                
                L_int.append(jnp.sum(out_vars[3], axis = 1))
                n_o_int.append(np.mean(out_vars[4][out_vars[1]]))
                N_int.append(jnp.sum(out_vars[4], axis = 1))
                n_o_std_int.append(np.std(out_vars[4][out_vars[1]]))
                pos_inn_int.append(np.mean(out_vars[1], axis=0))
                pos_obs_int.append(np.mean(out_vars[5], axis=0))
            
            fname = f'cache/L_comparison/{γ=}_{K=}_{dt=}_{G_in=}_{ro=}_{rd=}_{r=}_{I=}_SDE.p'
            with open(fname, 'wb') as f:
                pickle.dump({'G_in':G_in, 'r':r, 'rd':rd, 'ro':ro, 'I':I, 'el':el, 'γ':γ, 'K':K,
                         'samples':samples, 'tmax':tmax, 'save_dt':tplot, 'dt':dt,
                         'key':key, 'vars':out_vars_1, 'L':L, 'L_std':L_std, 'N':N, 'N_std':N_std, 'n_o':n_o, 'n_o_std':n_o_std, 'pos_inn': pos_inn, 'pos_obs': pos_obs},
                        f)
            print(f"Done with {fname}.")
            fname = f'cache/L_comparison/{γ=}_{K=}_{dt=}_{G_in=}_{ro=}_{rd=}_{r=}_{I=}_automaton.p'
            with open(fname, 'wb') as f:
                pickle.dump({'G_in':G_in, 'r':r, 'rd':rd, 'ro':ro, 'I':I, 'el':el, 'γ':γ, 'K':K,
                         'samples':samples, 'tmax':tmax, 'save_dt':tplot, 'dt':dt,
                         'key':key, 'vars':out_vars, 'L':L_int, 'L_std':L_std_int, 'N':N_int, 'N_std':N_std_int, 'n_o':n_o_int, 'n_o_std':n_o_std_int, 'pos_inn': pos_inn_int, 'pos_obs': pos_obs_int},
                        f)
            print(f"Done with {fname}.")

[1. 1. 1. ... 1. 1. 1.]
Done with cache/L_comparison/γ=0.0_K=2_dt=0.01_G_in=40_ro=0.1_rd=0.1_r=0.2_I=0.8_SDE.p.
Done with cache/L_comparison/γ=0.0_K=2_dt=0.01_G_in=40_ro=0.1_rd=0.1_r=0.2_I=0.8_automaton.p.
[1. 1. 1. ... 1. 1. 1.]
Done with cache/L_comparison/γ=0.5_K=2_dt=0.01_G_in=40_ro=0.1_rd=0.1_r=0.2_I=0.8_SDE.p.
Done with cache/L_comparison/γ=0.5_K=2_dt=0.01_G_in=40_ro=0.1_rd=0.1_r=0.2_I=0.8_automaton.p.
[1.  1.  1.  ... 0.5 1.  1. ]
Done with cache/L_comparison/γ=1.0_K=2_dt=0.01_G_in=40_ro=0.1_rd=0.1_r=0.2_I=0.8_SDE.p.
Done with cache/L_comparison/γ=1.0_K=2_dt=0.01_G_in=40_ro=0.1_rd=0.1_r=0.2_I=0.8_automaton.p.
[1. 1. 1. ... 1. 1. 1.]


In [18]:
for rd, ro in [(0.3, 0.3)]:
    #fig, ax = plt.subplots()
    for γ in [0, 0.5, 1]:
            γ = float(γ)
            fname = f'cache/L_comparison/{γ=}_{K=}_{dt=}_{G_in=}_{ro=}_{rd=}_{r=}_{I=}_automaton.p'
            print(f'Trying {fname}')
            with open(fname, 'rb') as f:
                algo = pickle.load(f)
            L = algo['L']
            L_std = algo['L_std']
            L_ = np.zeros(len(L)-1)
            L_std_ = np.zeros(len(L)-1)
            L_1 = np.zeros(len(L)-1)
            L_std_1 = np.zeros(len(L)-1)
            for i in range(len(L)-1):
                L_[i] = jnp.mean(L[i+1][L[i+1]>0])
                L_1[i] = jnp.mean(L[i+1])
                L_std_[i] = jnp.std(L[i+1][L[i+1]>0])
                L_std_1[i] = jnp.std(L[i+1])
            ax.fill_between(np.arange(0, len(L_))*1000*dt,
                        L_ - np.array(L_std_)/np.sqrt(samples),
                        L_ + np.array(L_std_)/np.sqrt(samples), color = colors[γ], alpha=0.5)
            if ro==0.2:
                ax.plot(np.arange(0, len(L_)),L_, color = colors[γ], label = f'{γ=}')
            else:
                ax.plot(np.arange(0, len(L_))*1000*dt,L_, color = colors[γ]) 
            fname = f'cache/L_comparison/{γ=}_{K=}_{dt=}_{G_in=}_{ro=}_{rd=}_{r=}_{I=}_SDE.p'
            print(f'Trying {fname}')
            with open(fname, 'rb') as f:
                algo = pickle.load(f)
            L = algo['L']
            L_std = algo['L_std']
            L_ = np.zeros(len(L)-1)
            L_std_ = np.zeros(len(L)-1)
            L_1 = np.zeros(len(L)-1)
            L_std_1 = np.zeros(len(L)-1)
            for i in range(len(L)-1):
                L_[i] = jnp.mean(L[i+1][L[i+1]>0])
                L_1[i] = jnp.mean(L[i+1])
                L_std_[i] = jnp.std(L[i+1][L[i+1]>0])
                L_std_1[i] = jnp.std(L[i+1])
            ax.fill_between(np.arange(0, len(L_))*1000*dt,
                        L_ - np.array(L_std_)/np.sqrt(samples),
                        L_ + np.array(L_std_)/np.sqrt(samples), color = colors[γ], alpha=0.5)
            if ro==0.2:
                ax.plot(np.arange(0, len(L_)),L_, color = colors[γ], label = f'{γ=}')
            else:
                ax.plot(np.arange(0, len(L_))*1000*dt,L_, color = colors[γ])              
                #ax.hlines(N_int, 100, (len(L_)+1) * save_dt * dt, ls =':',color=colors[γ])
                #ode2_net = ODE2Tree_obs_random( G=G_in/r*K, ro= ro/r, rd = rd/r, I = I, gamma=γ, k=K, L=None, alpha=1., Q=2)
                #L_int = G_in/(rd-ro*(1+γ*(K-1)))
                #ax.hlines(L_int, 100, (len(L_)+1) * save_dt * dt, ls =':',color=colors[γ])
                #ax.hlines(ode2_net.L, 100, (len(L_)+1) * save_dt * dt, ls =':',color=colors[γ])
            #ax.plot(L_1, color='g')
            if ro == ro:
                ode2_net = ODE2Tree_obs_random(G=G_in/(r), ro= ro/r, rd = rd/r, I = I, gamma=γ, k=K, L=None, alpha=1., Q=2)
                L_int2 = ode2_net.L
                if rd+ro*(1+γ*(K-1))-2*r> 0:
                    L_int = G_in*r*I/(K*(ro*(rd+ro*(1+γ*(K-1))-2*r)))
                else:
                    L_int = 2000
                if rd+ro/L_int-r >0 and G_in > ro/I:
                    N_int = (G_in - ro/I)/(rd+ro*(1+γ*(K-1))/L_int - r)
                else:
                    N_int = 0
                ax.hlines(L_int2, 0, (len(L_)+1)*1000*dt, ls =':',color=colors[γ])
            #if γ==0:
            #    ax1.plot([], shapes[ro], color = 'black', label = f'{ro=}_{rd=}')

            

    ax.set(title=f'Branching ratio Q ={K}', xlabel='Time', ylabel=r'System Size $L$')
    #axins.set()
    #axins.set_xticks([])
    #ax1.plot([], ls=':', color = 'black', label = r'Analytical')
    #ax1.set_yticks([])
    ax.legend(loc= 'upper left', fontsize ='10')
    #ax1.legend(loc= 'center left', fontsize ='10')
    #ax.set_ylim([0, 500])
    #plt.axis(ymin=0, ymax=50)
    #plt.tight_layout()
    #plt.show()
#fig.savefig("L_vs_time_g_ro.png", bbox_inches='tight')

Trying cache/L_comparison/γ=0.0_K=2_dt=0.01_G_in=40_ro=0.3_rd=0.3_r=0.2_I=0.8_automaton.p
Trying cache/L_comparison/γ=0.0_K=2_dt=0.01_G_in=40_ro=0.3_rd=0.3_r=0.2_I=0.8_SDE.p
Trying cache/L_comparison/γ=0.5_K=2_dt=0.01_G_in=40_ro=0.3_rd=0.3_r=0.2_I=0.8_automaton.p
Trying cache/L_comparison/γ=0.5_K=2_dt=0.01_G_in=40_ro=0.3_rd=0.3_r=0.2_I=0.8_SDE.p
Trying cache/L_comparison/γ=1.0_K=2_dt=0.01_G_in=40_ro=0.3_rd=0.3_r=0.2_I=0.8_automaton.p
Trying cache/L_comparison/γ=1.0_K=2_dt=0.01_G_in=40_ro=0.3_rd=0.3_r=0.2_I=0.8_SDE.p


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
